# Bài tập 2: Mô phỏng Hàng đợi Printer Spooler trong mạng LAN

**Mục tiêu:** Cài đặt mô phỏng hệ thống hàng đợi in (Printer Spooler) trong mạng LAN.

**Yêu cầu:**
- Các máy tính gửi yêu cầu in (print job) đến máy in
- Máy in xử lý lần lượt theo thứ tự FIFO (First In First Out)
- Mô phỏng hai trường hợp bộ nhớ:
  - **Bộ nhớ lớn:** Queue không giới hạn
  - **Bộ nhớ nhỏ:** Queue chỉ chứa tối đa 10 job (job thừa sẽ bị DROP)
- Thời gian in mỗi trang: 3-5 giây
- Số trang mỗi job: ngẫu nhiên từ 1-50 trang

## 1. Phân tích bài toán

### 1.1. Mô hình Producer-Consumer

Bài toán Printer Spooler là ví dụ điển hình của mô hình **Producer-Consumer**:

| Thành phần | Vai trò | Tốc độ |
|------------|---------|--------|
| **Producer** | Các máy tính gửi yêu cầu in | Nhanh (~0.5s/job) |
| **Consumer** | Máy in xử lý các job | Chậm (3-5s/job) |
| **Buffer** | Hàng đợi (Queue) chứa job chờ xử lý | Có thể giới hạn hoặc không |

### 1.2. Sơ đồ luồng dữ liệu

```
┌─────────┐     ┌─────────┐     ┌─────────┐
│ User 1  │────▶│         │     │         │
├─────────┤     │  QUEUE  │────▶│ PRINTER │────▶ Output
│ User 2  │────▶│ (Spool) │     │         │
├─────────┤     │         │     │         │
│ User N  │────▶│         │     │         │
└─────────┘     └─────────┘     └─────────┘
  (Fast)         (Buffer)         (Slow)
```

### 1.3. Hai trường hợp bộ nhớ

| Trường hợp | Mô tả | Xử lý khi đầy |
|------------|-------|---------------|
| **Unbounded Queue** | Không giới hạn số job | Luôn nhận job mới |
| **Bounded Queue** | Giới hạn N job (ví dụ: 10) | DROP job mới nếu đầy |

### 1.4. Tại sao cần đa luồng (Multi-threading)?

- **Vấn đề:** Nếu chạy tuần tự, khi máy in đang xử lý (sleep 5s), chương trình bị block, không thể nhận job mới
- **Giải pháp:** Sử dụng 2 luồng chạy song song:
  - **Luồng Producer:** Liên tục tạo và gửi job
  - **Luồng Consumer:** Liên tục lấy job từ queue và xử lý

### 1.5. Tại sao dùng `queue.Queue` thay vì `list`?

| Đặc điểm | `list` | `queue.Queue` |
|----------|--------|---------------|
| Thread-safe | ❌ Không | ✅ Có |
| Blocking get/put | ❌ Không | ✅ Có |
| Hỗ trợ maxsize | ❌ Không | ✅ Có |

`queue.Queue` có cơ chế **locking** tích hợp, đảm bảo an toàn khi nhiều luồng truy cập đồng thời.

---

## 2. Cài đặt chương trình

In [ ]:
import threading
import time
import random
import queue
from datetime import datetime

# ============================================================
# 2.1. ĐỊNH NGHĨA CLASS PRINTJOB
# ============================================================

class PrintJob:
    """
    Class đại diện cho một yêu cầu in.
    
    Attributes:
        job_id: Mã định danh của job
        owner: Tên người/phòng ban gửi yêu cầu
        pages: Số trang cần in (1-50)
        created_at: Thời điểm tạo job
    """
    
    def __init__(self, job_id, owner, pages):
        self.job_id = job_id
        self.owner = owner
        self.pages = pages
        self.created_at = datetime.now().strftime("%H:%M:%S")

    def __str__(self):
        return f"Job #{self.job_id} | {self.owner} | {self.pages} trang | {self.created_at}"


# ============================================================
# 2.2. HÀM MÔ PHỎNG MÁY IN (CONSUMER)
# ============================================================

def printer_worker(spool_queue, stop_event, stats):
    """
    Hàm chạy trong luồng riêng, mô phỏng máy in xử lý các job.
    
    Args:
        spool_queue: Hàng đợi chứa các PrintJob
        stop_event: Event để dừng máy in
        stats: Dict lưu thống kê
    """
    print("PRINTER: Máy in đã khởi động, đang chờ lệnh...\n")
    
    while not stop_event.is_set() or not spool_queue.empty():
        try:
            # Lấy job từ queue (timeout 1s để kiểm tra stop_event)
            job = spool_queue.get(timeout=1)
            
            # Bắt đầu in
            print(f"  [IN] Đang in: {job}")
            
            # Mô phỏng thời gian in (3-5 giây theo đề bài)
            print_time = random.uniform(3, 5)
            time.sleep(print_time)
            
            # Hoàn thành
            print(f"  [OK] Hoàn thành: Job #{job.job_id} - {print_time:.2f}s")
            
            # Cập nhật thống kê
            stats['completed'] += 1
            stats['total_pages'] += job.pages
            stats['total_time'] += print_time
            
            # Báo queue đã xử lý xong job
            spool_queue.task_done()
            
        except queue.Empty:
            # Queue rỗng, tiếp tục chờ
            continue
    
    print("\nPRINTER: Máy in đã tắt.")


# ============================================================
# 2.3. HÀM MÔ PHỎNG GỬI YÊU CẦU IN (PRODUCER)
# ============================================================

def send_print_jobs(spool_queue, num_jobs, stats):
    """
    Mô phỏng các máy tính trong LAN gửi yêu cầu in.
    
    Args:
        spool_queue: Hàng đợi để gửi job
        num_jobs: Số lượng job cần gửi
        stats: Dict lưu thống kê
    """
    # Danh sách người dùng/phòng ban
    users = ["Admin", "HR_Dept", "Dev_Team", "Manager", "Accounting", "Reception"]
    
    for i in range(1, num_jobs + 1):
        # Tạo job với số trang ngẫu nhiên 1-50
        pages = random.randint(1, 50)
        owner = random.choice(users)
        job = PrintJob(i, owner, pages)
        
        print(f"[->] {owner} gửi yêu cầu in {pages} trang...")
        
        try:
            # put_nowait: Thử đưa vào queue, nếu đầy thì raise Full
            spool_queue.put_nowait(job)
            print(f"     Đã vào hàng đợi (size: {spool_queue.qsize()})")
            stats['accepted'] += 1
            
        except queue.Full:
            # Queue đầy -> DROP job
            print(f"     [DROP] Hàng đợi đầy! Job #{i} bị hủy.")
            stats['dropped'] += 1
        
        # Mô phỏng tốc độ gửi nhanh (0.5s/job)
        time.sleep(0.5)


# ============================================================
# 2.4. HÀM CHÍNH CHẠY MÔ PHỎNG
# ============================================================

def run_simulation(max_queue_size=None, num_jobs=15):
    """
    Chạy mô phỏng Printer Spooler.
    
    Args:
        max_queue_size: Kích thước tối đa của queue (None = không giới hạn)
        num_jobs: Số lượng job gửi đến
    """
    # Thống kê
    stats = {
        'accepted': 0,      # Số job được nhận
        'dropped': 0,       # Số job bị DROP
        'completed': 0,     # Số job hoàn thành
        'total_pages': 0,   # Tổng số trang đã in
        'total_time': 0     # Tổng thời gian in
    }
    
    # Tạo queue
    if max_queue_size:
        spool_queue = queue.Queue(maxsize=max_queue_size)
        mode = f"BỘ NHỚ GIỚI HẠN ({max_queue_size} slot)"
    else:
        spool_queue = queue.Queue()  # Không giới hạn
        mode = "BỘ NHỚ KHÔNG GIỚI HẠN"
    
    print("=" * 60)
    print(f"MÔ PHỎNG PRINTER SPOOLER - {mode}")
    print(f"Số job gửi đến: {num_jobs}")
    print("=" * 60 + "\n")
    
    # Event để dừng máy in
    stop_event = threading.Event()
    
    # Khởi động luồng máy in (Consumer)
    printer_thread = threading.Thread(
        target=printer_worker, 
        args=(spool_queue, stop_event, stats)
    )
    printer_thread.start()
    
    # Gửi các job (Producer)
    send_print_jobs(spool_queue, num_jobs, stats)
    
    # Chờ queue xử lý hết
    spool_queue.join()
    
    # Dừng máy in
    stop_event.set()
    printer_thread.join()
    
    # In thống kê
    print("\n" + "=" * 60)
    print("THỐNG KÊ KẾT QUẢ")
    print("=" * 60)
    print(f"  Tổng job gửi đến:    {num_jobs}")
    print(f"  Job được nhận:       {stats['accepted']}")
    print(f"  Job bị DROP:         {stats['dropped']}")
    print(f"  Job hoàn thành:      {stats['completed']}")
    print(f"  Tổng số trang in:    {stats['total_pages']}")
    print(f"  Tổng thời gian in:   {stats['total_time']:.2f}s")
    if stats['completed'] > 0:
        print(f"  Thời gian TB/job:    {stats['total_time']/stats['completed']:.2f}s")
    print("=" * 60)

---

## 3. Chạy mô phỏng

### 3.1. Trường hợp 1: Bộ nhớ nhỏ (Queue giới hạn 10 slot)

Với queue chỉ chứa tối đa 10 job, khi có nhiều yêu cầu gửi đến nhanh hơn tốc độ xử lý, các job thừa sẽ bị **DROP**.

In [ ]:
# Chạy mô phỏng với bộ nhớ nhỏ (10 slot)
# Gửi 20 job để thấy hiện tượng DROP
run_simulation(max_queue_size=10, num_jobs=20)

### 3.2. Trường hợp 2: Bộ nhớ lớn (Queue không giới hạn)

Với queue không giới hạn, tất cả các job đều được nhận vào hàng đợi và xử lý tuần tự.

In [ ]:
# Chạy mô phỏng với bộ nhớ lớn (không giới hạn)
run_simulation(max_queue_size=None, num_jobs=10)

---

## 4. Phân tích kết quả

### 4.1. So sánh hai trường hợp

| Tiêu chí | Bộ nhớ nhỏ (10 slot) | Bộ nhớ lớn |
|----------|---------------------|------------|
| **Job bị DROP** | Có (khi queue đầy) | Không |
| **Độ tin cậy** | Thấp hơn | Cao hơn |
| **Sử dụng RAM** | Cố định, an toàn | Có thể tăng không kiểm soát |
| **Phù hợp** | Máy in giá rẻ, embedded | Server, máy in cao cấp |

### 4.2. Ví dụ trace với queue 3 slot

| Thời gian | Hành động | Queue | Ghi chú |
|-----------|-----------|-------|---------|
| T=0s | Job 1 đến | `[1]` | Máy in bắt đầu in Job 1 (cần 4s) |
| T=0.5s | Job 2 đến | `[2]` | Job 2 vào queue chờ |
| T=1s | Job 3 đến | `[2,3]` | Job 3 vào queue chờ |
| T=1.5s | Job 4 đến | `[2,3,4]` | Queue đầy (3/3) |
| T=2s | Job 5 đến | `[2,3,4]` | **DROP Job 5** |
| T=4s | Job 1 xong | `[3,4]` | Job 2 bắt đầu in, queue trống 1 slot |
| T=4.5s | Job 6 đến | `[3,4,6]` | Job 6 vào được (có slot trống) |

---

## 5. Tổng kết

- **Queue (Hàng đợi):** Cấu trúc FIFO để quản lý thứ tự xử lý
- **Multi-threading:** Cho phép Producer và Consumer chạy song song
- **Bounded Buffer:** Giới hạn kích thước queue để bảo vệ bộ nhớ